In [45]:
import re
import numpy as np
import pandas as pd
from collections import defaultdict
from profanity_check import predict, predict_prob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#parameters
MIN_COMMENT_LENGTH = 4;   #Flags any comment that has less than # characters
MIN_COMMENT_WORDS = 2;    #Flags any comment that has less than # words
PROFANITY_PROB = 0.5;     #Flags any comment that has profanity score greater than #

#load dataset 
filename = './dataset/YouTube-Spam-Collection/Youtube05-Shakira.csv'
data = pd.read_csv(filename)
data.head()
comments = np.array(data['CONTENT'].tolist())

dsgsdg

In [46]:
def length_filter(comments):
    """
    length based filter
    """
    flags_len=[]
    for comment in comments: 
        if len(comment) <= MIN_COMMENT_LENGTH:
            flags_len.append(True)
        else:
            flags_len.append(False)
    return flags_len 
    

print(comments[length_filter(comments)])

['wow' ':)' 'wow' ':)' 'Wow' 'cool' 'WOw' 'Nice' 'wow' 'wow']


In [47]:
def profanity_filter(comments):
    """
    profanity filter
    """
    flags_profanity = predict_prob(comments) > PROFANITY_PROB
    return flags_profanity


print(comments[profanity_filter(comments)])

['Fuck it was the best ever 0687119038 nummber of patrik kluivert his son share !\ufeff'
 'this song sucks\ufeff' 'THIS IS SHIT AND SOOOOO AUTOTUNED\ufeff'
 'Message :   GTA V  $20  FIFA 14 $15  PS4  $200  Galaxy S4 mini $250  Ipad 4   $200  visit the site hh.nl'
 'Your a fucking bitch'
 '  Perhaps you have seen the newest Miley Cyrus SECRET video ?   She&#39;s sucking an old man&#39;s cock ,  If you wish to see her , check out the celebrity website beneath :   miley-secret-video.co.uk '
 ' I really can&#39;t comprehend Miley Cyrus , she actually is a high profile and she tapes herself banging Today a video was leeched with her sucking and fucking The video has been posted at the celebrity website under :  miley-celeb-news.co.uk '
 'WOW muslims are really egoistic..... 23% of the World population and not in this video or donating 1 dollar to the poor ones in Africa :( shame on those terrorist muslims'
 'Hey, hit this shit up while yall can, they killed the versace remix. Just type in C

In [54]:
def dict_filter(comments):
    """
    dictionary based filter
    """
    path1 = './dataset/hindi_dict.txt'
    path2 = './dataset/english_dict.txt'
    path3 = './dataset/spam_ads_dict.txt'
    path4 = './dataset/blacklist_dict.txt'
    with open(path1) as f1, open(path2) as f2, open(path3) as f3, open(path4) as f4:
        content_hi = f1.read()
        content_en = f2.read()
        content_spam = f3.read()
        content_black = f4.read()

    words_hi = content_hi.split("\n")
    words_en = content_en.split("\n")
    words_spam = content_spam.split("\n")
    words_black = content_black.split("\n")

    filtered_comments = defaultdict(list)
    
    for comment in comments: 
        for word in words_hi:
            pattern = re.compile(word)
            match1 = pattern.search(comment.lower())
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        filtered_comments['hindi'].append(comment)
                        #print("{:-<20} {}".format(word, comment))

    #nsfw english filter
    filter_words_en = []
    for comment in comments: 
        for word in words_en:
            pattern = re.compile(word)
            match1 = pattern.search(comment.lower())
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        filtered_comments['english'].append(comment)
                        #print("{:-<20} {}".format(word, comment))    
                 
    #spam filter
    for comment in comments: 
        for word in words_spam:
            pattern = re.compile(word)
            match1 = pattern.search(comment.lower())
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        filtered_comments['spam'].append(comment)
                        #print("{:-<20} {}".format(word, comment))
                        

    #blacklist filter
    for comment in comments: 
        for word in words_black:
            pattern = re.compile(word)
            match1 = pattern.search(comment.lower())
            if match1 is not None :
                for w in comment.split(' '):
                    match2 = re.fullmatch(pattern, w.lower())
                    if match2 is not None :
                        filtered_comments['blacklist'].append(comment)
                        #print("{}".format(comment))
                        
    return filtered_comments

dict_filter(comments)['spam']


['I really love watching football and also I’ve started off making income with out financial risk from acquiring bonus deals. It’s this weird technique where you wager on something with one bookmakers and bet against it on Betfair. You secure the bonus as income . A chap named Jim Vanstone is finding the wagers free on his own website Vanstone Secrets (Google it!). I’ve made about 500 quid thus far. And it is cost-free. I guess the bookies pay him to obtain new consumers, yet this actually works.',
 'I really love watching football and also I’ve started off making income with out financial risk from acquiring bonus deals. It’s this weird technique where you wager on something with one bookmakers and bet against it on Betfair. You secure the bonus as income . A chap named Jim Vanstone is finding the wagers free on his own website Vanstone Secrets (Google it!). I’ve made about 500 quid thus far. And it is cost-free. I guess the bookies pay him to obtain new consumers, yet this actually w

In [64]:
def senti_filter(comments):
    """
    sentiment based filter
    """
    flags_senti = []
    analyzer = SentimentIntensityAnalyzer()  
    for comment in comments: 
        score = analyzer.polarity_scores(comment)
        if score['neg'] > 0.2:
            flags_senti.append(True)
        else:
            flags_senti.append(False)

    return comments[flags_senti]

senti_filter(comments)

# sentiment_analyzer = SentimentIntensityAnalyzer()    
# score = sentiment_analyzer.polarity_scores("this is fucking amazing")
# print(score)

# for comment in dict_filter(comments)['english']: 
#     score = sentiment_analyzer.polarity_scores(comment)
#     if score['compound'] > 0:
#         print(comment, score)



array(['Why there are so many dislikes. This song is so... awesome. It sounds like we MUST STOP BE RACISTS!!! If I could, I would like it 1,000,000,000 times!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\ufeff',
       'I really am madly in love with this woman!!\ufeff',
       'How could 108k people dislike this song or video\ufeff',
       'it pisses me off a bit that blank space has more views (868 mio) than this. meh \ufeff',
       'I believe that soccer promotes terrorism. Bad choice, Shakira.\ufeff',
       '5 years ago damn \ufeff',
       'So underrated better<br />Than  Katy perry :/ but in not saying Katy is bad but she has no meaning to her songs. Shakira has<br />Meaningful songs these are the songs which bring memories and vibe&#39;s. I will miss it.   <br />   \ufeff',
       'Hi, nice song Shakira! (Sorry for bad Brazilian)\ufeff',
       'THIS IS SHIT AND SOOOOO AUTOTUNED\ufeff', 'Your a fucking bitch',
       'this song is racist', 'This makes me miss the world cup',
